# Metaklasy

In [1]:
class Shape:
    def draw(self):
        pass

class Rectangle(Shape):
    _id = 'RECT'

    def __init__(self, width, height):
        self.width = width
        self.height = height        

    def draw(self):
        print(f'Drawing {Rectangle._id}({self.width}, {self.height})')

In [4]:
r = Rectangle(100, 200)
r.draw()

Drawing RECT(100, 200)


In [5]:
type(Rectangle)

type

In [6]:
Rectangle.__name__

'Rectangle'

To samo ale dynamicznie:

In [7]:
def rect_init(self, width, height):
    self.width = width
    self.height = height 

RectangleT = type('RectangleT', (Shape, ), {
    '_id': 'RECT_T',
    '__init__': rect_init,
    'draw': lambda self: print(f'Drawing {RectangleT._id}({self.width}, {self.height})')
})

In [8]:
type(RectangleT)

type

In [9]:
RectangleT.__name__

'RectangleT'

In [10]:
RectangleT.__mro__

(__main__.RectangleT, __main__.Shape, object)

In [11]:
rt = RectangleT(100, 200)

In [12]:
rt.draw()

Drawing RECT_T(100, 200)


In [13]:
class ColorRectT(RectangleT):
    def __init__(self, w, h, color):
        super().__init__(w, h)
        self.color = color

In [14]:
color_rect = ColorRectT(100, 200, 255)

In [15]:
color_rect.width

100

In [16]:
color_rect.color

255

## Metaclass

In [ ]:
class Metaclass(type):
    @classmethod 
    def __prepare__(mcs, name, bases, **kwargs): 
        return super().__prepare__(name, bases, **kwargs)

    def __new__(mcs, name, bases, class_dict):
        return super().__new__(mcs, name, bases, class_dict)
    
    def __init__(cls, name, bases, class_dict, **kwargs): 
        super().__init__(name, bases, class_dict) 

    def __call__(cls, *args, **kwargs): 
        return super().__call__(*args, **kwargs)

In [18]:
class RevealingMeta(type): 
    def __new__(mcs, name, bases, namespace, **kwargs): 
        print(mcs, "METACLASS __new__ called") 
        return super().__new__(mcs, name, bases, namespace) 
 
    @classmethod 
    def __prepare__(mcs, name, bases, **kwargs): 
        print(mcs, " METACLASS __prepare__ called") 
        return super().__prepare__(mcs, name, bases, **kwargs) 
 
    def __init__(cls, name, bases, namespace, **kwargs): 
        print(cls, " METACLASS __init__ called") 
        super().__init__(name, bases, namespace) 
 
    def __call__(cls, *args, **kwargs): 
        print(cls, " METACLASS __call__ called") 
        return super().__call__(*args, **kwargs) 

In [19]:
class RevealingClass(metaclass=RevealingMeta):
    def __new__(cls):
        print(cls, "__new__ called")
        return super().__new__(cls)
    
    def __init__(self):
        print(self, "__init__ called")
        super().__init__()

<class '__main__.RevealingMeta'>  METACLASS __prepare__ called
<class '__main__.RevealingMeta'> METACLASS __new__ called
<class '__main__.RevealingClass'>  METACLASS __init__ called


In [20]:
instance_rc = RevealingClass()

<class '__main__.RevealingClass'>  METACLASS __call__ called
<class '__main__.RevealingClass'> __new__ called
<__main__.RevealingClass object at 0x000001EC196F9510> __init__ called


In [17]:
from typing import Any, Mapping
import inflection

class CaseInterpolationDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, value)
        super().__setitem__(inflection.underscore(key), value)

class CaseInterpolatedMeta(type):
    """metaclass"""
    @classmethod
    def __prepare__(metacls, __name: str, __bases: tuple[type, ...], **kwds: Any) -> Mapping[str, object]:
        return CaseInterpolationDict()

In [24]:
class MyUser(metaclass=CaseInterpolatedMeta):
    pass

class User(MyUser):
    idUser = 0

    def __init__(self, firstName: str, lastName: str):
        self.firstName = firstName
        self.lastName = lastName

    def getDisplayName(self):
        return f"{self.firstName} {self.lastName}"
    
    def greetUser(self):
        return f"Hello {self.getDisplayName()}!"

In [25]:
admin = User("admin", "root")

In [26]:
admin.getDisplayName()

AttributeError: 'User' object has no attribute 'getDisplayName'

In [27]:
admin.get_display_name()

'admin root'

In [28]:
User.id_user

0

# Metaprogramming

# How it works - namedtuple

In [30]:
from typing import Iterable, Iterator, Union


FieldNames = Union[str, Iterable[str]]


def parse_identifiers(names: FieldNames) -> tuple[str, ...]:
    if isinstance(names, str):
        names = names.replace(',', ' ').split()

    if not all(s.isidentifier() for s in names):
        raise ValueError('names must all be valid identifiers')

    return tuple(names)

def record_factory(cls_name, field_names: FieldNames):
    slots = parse_identifiers(field_names)

    def __init__(self, *args, **kwargs) -> None:
        attrs = dict(zip(self.__slots__, args))
        attrs.update(kwargs)

        for name, value in attrs.items():
            setattr(self, name, value)

    def __iter__(self) -> Iterator[Any]:
        for name in self.__slots__:
            yield getattr(self, name)

    def __repr__(self):
        values = ', '.join(f"{name}={value!r}" for name,
                           value in zip(self.__slots__, self))
        cls_name = self.__class__.__name__
        return f"{cls_name}({values})"

    cls_attrs = dict(
        __slots__ = slots,
        __init__ = __init__,
        __repr__ = __repr__,
        __iter__ = __iter__
    )

    return type(cls_name, (object, ), cls_attrs)

In [31]:
Point = record_factory('Point', 'x y')

In [32]:
pt1 = Point(10, 20)

In [33]:
pt1.x

10

In [34]:
pt1

Point(x=10, y=20)

In [35]:
for i in pt1:
    print(i)

10
20


## 

## NamedTuple

In [38]:
class MetaBunch(type):
    def __new__(mcl, classname, bases, classdict):
        
        # Define as local functions __init__ & __repr__ that we'll use 
        # in the new class
        def __init__(self, **kwargs):
            """__init__ is simple: first, set attributes without
               explicit values to their defaults; then, set those
               explicitly passed in kw.
            """
            for k in self.__dflts__:
                if not k in kwargs:
                    setattr(self, k, self.__dflts__[k])
            for k in kwargs:
                setattr(self, k, kwargs[k])

        def __repr__(self):
            """__repr__ is minimal: shows only attributes that
               differ from default values, for compactness.
            """
            rep = [f'{k}={getattr(self, k)!r}' for k in self.__dflts__ if getattr(self, k) != self.__dflts__[k]]
            return f"{classname}({', '.join(rep)})"
        
        newdict = { 
            '__slots__': [], 
            '__dflts__': {},
            '__init__' : __init__, 
            '__repr__' : __repr__,
        }
        
        for k in classdict:
            if k.startswith('__') and k.endswith('__'):
                if k in newdict:
                    warnings.warn(f"Cannot set attr {k!r} in bunch-class {classname!r}")
                else:
                    newdict[k] = classdict[k]
            else:
                newdict['__slots__'].append(k)
                newdict['__dflts__'][k] = classdict[k]

        return super().__new__(mcl, classname, bases, newdict)
    
class Bunch(metaclass=MetaBunch):
    pass

In [39]:
class Point(Bunch):
    x = 0
    y = 0
    color = 'red'

pt = Point(x=10, y=20)

In [40]:
pt.x

10

In [41]:
pt.color

'red'

In [42]:
pt1

Point(x=10, y=20)

## ABC

In [45]:
def abstractmethod(function):
    function.__abstract__ = True
    return function 

In [46]:
import functools


class AbstractMeta(type):
    def __new__(metaclass, name, bases, namespace):
        # Create the class instance
        print(f"__new__({metaclass}, {name}, {bases!r}, {namespace!r})")
        cls = super().__new__(metaclass, name, bases, namespace)

        # Collect all local method marked as abstract
        abstracts = set()
        for k, v in namespace.items():
            if getattr(v, '__abstract__', False):
                abstracts.add(k)

        # Look for abstract methods in the base classes and add them to the list of abstracts
        for base in bases:
            for k in getattr(base, '__abstracts__', ()):
                v = getattr(cls, k, None)
                if getattr(v, '__abstract__', False):
                    abstracts.add(k)

        # Store the abstracts in a frozenset so the cannot be modified
        cls.__abstracts__ = frozenset(abstracts)

        # Decorate the __new__ function to check if all abstract functions were implemented
        original_new = cls.__new__
        @functools.wraps(original_new)
        def new(self, *args, **kwargs):
            for k in self.__abstracts__:
                v = getattr(self, k)
                if getattr(v, '__abstract__', False):
                    raise RuntimeError(f'{k} is not implemented')
            return original_new(self, *args, **kwargs)
        
        cls.__new__ = new
        return cls
    
class MyAbc(metaclass=AbstractMeta):
    pass

__new__(<class '__main__.AbstractMeta'>, MyAbc, (), {'__module__': '__main__', '__qualname__': 'MyAbc'})


In [48]:
class Shape(MyAbc):

    @abstractmethod
    def draw(self) -> None:
        pass


class RotableShape(Shape):
    @abstractmethod
    def rotate(self, angle):
        pass

    def draw(self):
        print("Drawing shape")

class Rectangle(RotableShape):
    pass
    # def rotate(self, angle):
    #     print("Rotate rectangle")

r = Rectangle()

__new__(<class '__main__.AbstractMeta'>, Shape, (<class '__main__.MyAbc'>,), {'__module__': '__main__', '__qualname__': 'Shape', 'draw': <function Shape.draw at 0x000002A168CAA710>})
__new__(<class '__main__.AbstractMeta'>, RotableShape, (<class '__main__.Shape'>,), {'__module__': '__main__', '__qualname__': 'RotableShape', 'rotate': <function RotableShape.rotate at 0x000002A168CAA170>, 'draw': <function RotableShape.draw at 0x000002A168CAAA70>})
__new__(<class '__main__.AbstractMeta'>, Rectangle, (<class '__main__.RotableShape'>,), {'__module__': '__main__', '__qualname__': 'Rectangle'})


RuntimeError: rotate is not implemented

In [49]:
Rectangle.__abstracts__

frozenset({'rotate'})

In [50]:
Shape.__abstracts__

frozenset({'draw'})